# Putting it all together using LangGraph (experimental)

**Last notebook we created a brain langchain agent that acts as a supervisor of the experts (other agents with tools).** This concept of multi-agent architectures is a current field of high research interest.

**Graphs are important in multi-agent systems** as they efficiently represent the interactions and relationships between different agents:
- **Nodes (or Vertices):** Each agent, can perform specific tasks or make decisions.
- **Edges:** Signify communication paths or decision flows between agents.

This structure enables the division of complex problems into smaller, manageable tasks, where each agent can focus on a particular aspect. The advantages of this approach include:
- Improved specialization and parallelization.
- More robust and scalable solutions.

**In our context, we can use the graph-based architecture** to build systems where agents coordinate through a defined protocol of interactions, enhancing both performance and flexibility. For instance, if one agent fails or generates suboptimal results, other agents in the system can take over or adjust their strategies accordingly, thus ensuring continuity and efficiency.

[**LangGraph**](https://python.langchain.com/docs/langgraph/), specifically designed for building multi-agent systems with LLMs, leverages these graph concepts by allowing developers to define states and transitions explicitly. It treats each agent as a state machine, with transitions defined by the possible actions an agent can take in response to its environment or the inputs it receives. LangGraph provides tools for building these systems, where each node in the graph can be an agent capable of making decisions or performing actions based on the shared state of the system. The library supports creating complex workflows where agents interact through a shared state, making it easier to develop, test, and maintain each component in isolation before integrating them into the broader system.

---


In [1]:
import os
import operator
from typing import TypedDict, Annotated, Sequence, Union, List
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.manager import CallbackManager

from langgraph.prebuilt import ToolExecutor, ToolNode
from langgraph.graph import StateGraph, MessageGraph, END


#custom libraries that we will use later in the app
from common.utils import (
    DocSearchAgent, 
    CSVTabularAgent, 
    SQLSearchAgent, 
    ChatGPTTool, 
    BingSearchAgent,
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT, CUSTOM_CHATBOT_PREFIX

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))


In [2]:
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Define our Model

In [3]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

# Uncomment below if you want to see the answers streaming
# llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, 
#                        max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager)


### Define the Tools

In [4]:
doc_indexes = ["srch-index-files", "srch-index-csv"]
doc_search = DocSearchAgent(llm=llm, indexes=doc_indexes,
                           k=6, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="docsearch",
                           description="useful when the questions includes the term: docsearch",
                           verbose=False)

In [5]:
book_indexes = ["srch-index-books"]
book_search = DocSearchAgent(llm=llm, indexes=book_indexes,
                           k=10, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="booksearch",
                           description="useful when the questions includes the term: booksearch",
                           verbose=False)

In [6]:
# BingSearchAgent is a langchain Tool class to use the Bing Search API (https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)
www_search = BingSearchAgent(llm=llm, k=5,
                             name="bing",
                             description="useful when the questions includes the term: bing",
                             verbose=False)

In [7]:
## CSVTabularAgent is a custom Tool class crated to Q&A over CSV files
file_url = "./data/all-states-history.csv"
csv_search = CSVTabularAgent(path=file_url, llm=llm,
                             name="csvfile",
                             description="useful when the questions includes the term: csvfile",
                             verbose=False)

In [8]:
## SQLDbAgent is a custom Tool class created to Q&A over a MS SQL Database
sql_search = SQLSearchAgent(llm=llm, k=30,
                            name="sqlsearch",
                            description="useful when the questions includes the term: sqlsearch",
                            verbose=False)

In [9]:
## ChatGPTTool is a custom Tool class created to talk to ChatGPT knowledge
chatgpt_search = ChatGPTTool(llm=llm,
                             name="chatgpt",
                            description="useful when the questions includes the term: chatgpt",
                            verbose=False)

In [10]:

tools = [www_search, sql_search, doc_search, csv_search, chatgpt_search, book_search]


### Bind the Tools to the Model

We should make sure the model knows that it has these tools available to call. We can do this by converting the LangChain tools into the format for OpenAI tool calling, and then bind them to the model class.

In [11]:
llm_with_tools = llm.bind_tools(tools) 

Let's test our model with a few messages to see if effectively calls the function sqlsearch

In [12]:
llm_with_tools.invoke([HumanMessage(content="what is your name"), 
              AIMessage(content='My name is Assistant. How can I assist you today?'),
              HumanMessage(content="sqlsearch, how many deaths in the east coast?")])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qj9So05P8CiSAj6XxaW7iz6c', 'function': {'arguments': '{"query":"how many deaths in the east coast"}', 'name': 'sqlsearch'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 430, 'total_tokens': 450}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-cf36f0cd-b62c-4081-878e-4c354e80139e-0', tool_calls=[{'name': 'sqlsearch', 'args': {'query': 'how many deaths in the east coast'}, 'id': 'call_qj9So05P8CiSAj6XxaW7iz6c'}], usage_meta

This looks correct!,  the LLM responded with a AIMessage that has no content, but instead has a **tool_call**.

### Define agent state

A graph is parameterized by a state object that it passes around to each node. Each node then returns operations to update that state. These operations can either SET specific attributes on the state (e.g. overwrite the existing values) or ADD to the existing attribute. Whether to set or add is denoted by annotating the state object you construct the graph with.

For our case, the state we will track will just be a list of messages. We want each node to just add messages to that list. Therefore, we will use a TypedDict with one key (messages) and annotate it so that the messages attribute is always added to with the second parameter (operator.add).

In [13]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

### Define the nodes

We now need to define a few different nodes in our graph. In langgraph, **a node can be either a function or a runnable**. There are two main nodes we need for this:

**The agent**: responsible for deciding what (if any) actions to take.
A function to invoke tools: if the agent decides to take an action, this node will then execute that action.
We will also need to define some edges. Some of these edges may be conditional. The reason they are conditional is that based on the output of a node, one of several paths may be taken. The path that is taken is not known until that node is run (the LLM decides).

**Conditional Edge**: after the agent is called, we should either:

    a. If the agent said to take an action, then the function to invoke tools should be called

    b. If the agent said that it was finished, then it should finish

**Normal Edge**: after the tools are invoked, it should always go back to the agent to decide what to do next

Let's define the nodes, as well as a function to decide how what conditional edge to take.

In [14]:
# Define the function that determines whether to continue or not
def should_continue(state):
    
    messages = state["messages"]
    
    last_message = messages[-1]
    # If there are no tool calls, then we finish
    if not last_message.tool_calls:
        return "end"
   # Otherwise if there is, we check if it's suppose to return direct the result of the tool or not (based on the prompt)
    else:
        arguments = last_message.tool_calls[0]["args"]
        if arguments.get("return_direct", False):
            return "final"
        else:
            return "continue"

    

# Define the function that calls the supervisor chain
async def supervisor_node(state):
    
    messages = state["messages"]
    
    PROMPT = ChatPromptTemplate.from_messages(
        [
            ("system", CUSTOM_CHATBOT_PREFIX),
            ("human", "{question}")
        ]
    )
    
    chain = (
        {
            "question": lambda x: x["question"],
        }
        | PROMPT
        | llm_with_tools
    )
    
    response = await chain.ainvoke({"question": messages})
    
    return {"messages": [response]}


# Define the function to execute tools
tool_node = ToolNode(tools)

### Define the Graph

In [15]:
# Define a new graph
workflow = StateGraph(AgentState)


# Define the two nodes we will cycle between
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("tools", tool_node)
# We add a separate node for any tool call where return_direct=True. The reason this is needed is that after this node we want to end, 
# while after other tool calls we want to go back to the LLM.
workflow.add_node("tools_final", tool_node)

# Set the entrypoint as `supervisor`
# This means that this node is the first one called
workflow.set_entry_point("supervisor")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `supervisor`.
    # This means these are the edges taken after the `supervisor` node is called.
    "supervisor",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "tools",
        # Final call
        "final": "tools_final",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `supervisor`.
# This means that after `tools` is called, `supervisor` node is called next.
workflow.add_edge("tools", "supervisor")
# and from `tools_final` to END
workflow.add_edge("tools_final", END)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

### Define a utility print function

In [16]:
async def print_events(query):
    inputs = {"messages": [HumanMessage(content=query)]}
    event_names = ["LangGraph", "supervisor", "call_supervisor_chain", "should_continue", "tools", "tools_final"]

    async for event in app.astream_events(inputs, version="v1"):
        kind = event["event"]
        # print(event)
        if kind == "on_chain_start":
            if event["name"] in event_names:
                print("\n=======================\n")
                print(f"Starting: {event['name']}")
                print(event)

        elif kind == "on_chain_end":
            if event["name"] == "LangGraph":  
                print("\n=======================\n")
                # print(event)
                try:
                    printmd(event['data']['output']['supervisor']['messages'][0].content)
                except:
                    print(event['data']['output'])
                    printmd(event['data']['output'][1]['tools_final']['messages'][0].content)
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            # Empty content in the context of OpenAI means that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            if content:
                print(content, end="|")
        elif kind == "on_tool_start":
            print("\n=======================\n")
            print(f"Starting tool: {event['name']}")
            print(event)

### Use the graph

In [17]:
await print_events("Hello there, how are you? My name is Pablo Marin")

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(




Starting: LangGraph
{'event': 'on_chain_start', 'run_id': '27923040-d0ca-4874-b51b-9bff0ce8ddec', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='Hello there, how are you? My name is Pablo Marin')]}}, 'parent_ids': []}


Starting: supervisor
{'event': 'on_chain_start', 'name': 'supervisor', 'run_id': 'c85f6126-5e16-4987-93e7-8ab6e1cab6d9', 'tags': ['graph:step:1'], 'metadata': {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_triggers': ['start:supervisor'], 'langgraph_task_idx': 0}, 'data': {}, 'parent_ids': []}


Starting: should_continue
{'event': 'on_chain_start', 'name': 'should_continue', 'run_id': 'd6ade7d2-06a3-4203-bdc7-67a15ffa846d', 'tags': ['seq:step:3'], 'metadata': {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_triggers': ['start:supervisor'], 'langgraph_task_idx': 0}, 'data': {'input': {'messages': [HumanMessage(content='Hello there, how are you? My name is Pablo Marin'), AIMessage(

Hello Pablo Marin! I'm Jarvis, your assistant. I'm here to help you with any questions or tasks you have. How can I assist you today?

In [18]:
await print_events("bing, What's the oldest parrot alive, and how much longer is that than the average lifespan of a parrot?")



Starting: LangGraph
{'event': 'on_chain_start', 'run_id': '429c72a4-0f32-4be7-b70b-95e2e883acc9', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content="bing, What's the oldest parrot alive, and how much longer is that than the average lifespan of a parrot?")]}}, 'parent_ids': []}


Starting: supervisor
{'event': 'on_chain_start', 'name': 'supervisor', 'run_id': 'f11c02d8-e966-4117-827a-13de54c7e36b', 'tags': ['graph:step:1'], 'metadata': {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_triggers': ['start:supervisor'], 'langgraph_task_idx': 0}, 'data': {}, 'parent_ids': []}


Starting: should_continue
{'event': 'on_chain_start', 'name': 'should_continue', 'run_id': '0e1e34e5-5bf4-4be7-aeb3-652bd6431ab2', 'tags': ['seq:step:3'], 'metadata': {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_triggers': ['start:supervisor'], 'langgraph_task_idx': 0}, 'data': {'input': {'messages': [HumanMessage(content="bing, 

The current oldest living parrot is **Poncho**, a Green-winged macaw, who is 92 years old. Poncho resides in the United States and was originally owned by Birds and Animals Unlimited. Most birds of Poncho's breed typically live to be around 50 or 60 years old, though some can reach 80 years with a healthy diet. Poncho has exceeded these expectations and is officially recognized as the world's oldest living parrot [[1]](https://www.oldest.org/animals/parrots/).

### Comparison to Average Lifespan
- **Poncho's Age:** 92 years
- **Average Lifespan of Green-winged Macaw:** 50-60 years (with some reaching up to 80 years)

**Difference:**
- Compared to the average lifespan of 50-60 years, Poncho has lived 32-42 years longer.
- Compared to the upper limit of 80 years, Poncho has lived 12 years longer.

### We can tell the app to return the results directly from the tool (without passing through the supervisor)

In [19]:
await print_events("docsearch, how diabetes affects covid? return the answer directly by setting return_direct=True")



Starting: LangGraph
{'event': 'on_chain_start', 'run_id': 'db898ed0-50eb-4ca9-a65a-59bcd91d57ea', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='docsearch, how diabetes affects covid? return the answer directly by setting return_direct=True')]}}, 'parent_ids': []}


Starting: supervisor
{'event': 'on_chain_start', 'name': 'supervisor', 'run_id': 'aa4cec6a-1f62-40bf-b432-b45bb4ee80f0', 'tags': ['graph:step:1'], 'metadata': {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_triggers': ['start:supervisor'], 'langgraph_task_idx': 0}, 'data': {}, 'parent_ids': []}


Starting: should_continue
{'event': 'on_chain_start', 'name': 'should_continue', 'run_id': '825dd388-34fe-491f-80f9-106251b02acc', 'tags': ['seq:step:3'], 'metadata': {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_triggers': ['start:supervisor'], 'langgraph_task_idx': 0}, 'data': {'input': {'messages': [HumanMessage(content='docsearch, how 

Diabetes significantly affects the progression and outcomes of COVID-19. Here are the key findings from various studies on this topic:

1. **Increased Severity and Mortality Risk**:
   - Patients with diabetes are at a higher risk of severe COVID-19 and have a doubled mortality risk due to complications involving the pulmonary and cardiac systems [[2]](https://doi.org/10.1007/s00508-020-01672-3?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-07-09T13:52:04Z&st=2024-07-09T05:52:04Z&spr=https&sig=9Nx31tWOzf6CWylUZnGaciT9VDWVJSJ9vQulMcshm7Q%3D).

2. **Inflammatory Response and Complications**:
   - COVID-19 can exacerbate complications in individuals with diabetes through an imbalance in angiotensin-converting enzyme 2 (ACE2) activation pathways, leading to an inflammatory response. This imbalance can cause acute β-cell dysfunction in the pancreas, resulting in a hyperglycemic state [[1]](https://api.elsevier.com/content/article/pii/S1056872720303962; https://www.sciencedirect.com/science/article/pii/S1056872720303962?v=s5?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-07-09T13:52:04Z&st=2024-07-09T05:52:04Z&spr=https&sig=9Nx31tWOzf6CWylUZnGaciT9VDWVJSJ9vQulMcshm7Q%3D).

3. **Biomarkers and Hypercoagulable State**:
   - Diabetic patients with COVID-19 show higher levels of inflammatory biomarkers such as IL-6, C-reactive protein, serum ferritin, and D-dimer, indicating they are more susceptible to an inflammatory storm. This can lead to rapid deterioration of their condition [[3]](https://doi.org/10.1002/dmrr.3319; https://www.ncbi.nlm.nih.gov/pubmed/32233013/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-07-09T13:52:04Z&st=2024-07-09T05:52:04Z&spr=https&sig=9Nx31tWOzf6CWylUZnGaciT9VDWVJSJ9vQulMcshm7Q%3D).

4. **Management Challenges**:
   - Managing diabetes during the pandemic is challenging due to limited access to outpatient clinics. This situation necessitates alternative treatment options like telemedicine. Proper glycemic control is crucial, and adjustments in antidiabetic drugs and insulin therapy may be required [[2]](https://doi.org/10.1007/s00508-020-01672-3?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-07-09T13:52:04Z&st=2024-07-09T05:52:04Z&spr=https&sig=9Nx31tWOzf6CWylUZnGaciT9VDWVJSJ9vQulMcshm7Q%3D).

5. **Preventive Measures**:
   - For individuals with diabetes, it is essential to stay hydrated, monitor blood glucose regularly, and check ketone bodies in urine if on insulin. Maintaining physical activity and a healthy diet are also crucial [[1]](https://api.elsevier.com/content/article/pii/S1056872720303962; https://www.sciencedirect.com/science/article/pii/S1056872720303962?v=s5?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-07-09T13:52:04Z&st=2024-07-09T05:52:04Z&spr=https&sig=9Nx31tWOzf6CWylUZnGaciT9VDWVJSJ9vQulMcshm7Q%3D).

In summary, diabetes significantly increases the risk of severe COVID-19 and mortality due to various complications, including heightened inflammatory responses and hypercoagulable states. Effective management and preventive measures are crucial for mitigating these risks.

In [20]:
await print_events("sqlsearch, How many people died of covid in Texas in 2020? return the answer directly by setting return_direct=True")



Starting: LangGraph
{'event': 'on_chain_start', 'run_id': '881137da-5c50-4b72-868a-150d4d121bf4', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='sqlsearch, How many people died of covid in Texas in 2020? return the answer directly by setting return_direct=True')]}}, 'parent_ids': []}


Starting: supervisor
{'event': 'on_chain_start', 'name': 'supervisor', 'run_id': '1a18bdc6-f62c-48d4-a9a4-ac4b1d05f2c1', 'tags': ['graph:step:1'], 'metadata': {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_triggers': ['start:supervisor'], 'langgraph_task_idx': 0}, 'data': {}, 'parent_ids': []}


Starting: should_continue
{'event': 'on_chain_start', 'name': 'should_continue', 'run_id': 'f7eacbbe-9d63-4217-b332-206b6f5c22bd', 'tags': ['seq:step:3'], 'metadata': {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_triggers': ['start:supervisor'], 'langgraph_task_idx': 0}, 'data': {'input': {'messages': [HumanMessage(cont

Final Answer: There were 2,841,253 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the sum of the `death` column where the state is 'TX' and the date starts with '2020'. The query returned a total of 2,841,253 deaths for the year 2020. 
I used the following query:

```sql
SELECT SUM(death) AS total_deaths FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'
```

In [21]:
await print_events("Thank you!! you are a great assistant. BTW, what is my name?")



Starting: LangGraph
{'event': 'on_chain_start', 'run_id': '1ac95127-f520-4e2c-aa48-c04ca5336bcd', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'messages': [HumanMessage(content='Thank you!! you are a great assistant. BTW, what is my name?')]}}, 'parent_ids': []}


Starting: supervisor
{'event': 'on_chain_start', 'name': 'supervisor', 'run_id': '1810a90f-b996-487c-bf8d-ddf708074de6', 'tags': ['graph:step:1'], 'metadata': {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_triggers': ['start:supervisor'], 'langgraph_task_idx': 0}, 'data': {}, 'parent_ids': []}


Starting: should_continue
{'event': 'on_chain_start', 'name': 'should_continue', 'run_id': '1a9f8884-895c-4235-82bb-e2ecbb49b082', 'tags': ['seq:step:3'], 'metadata': {'langgraph_step': 1, 'langgraph_node': 'supervisor', 'langgraph_triggers': ['start:supervisor'], 'langgraph_task_idx': 0}, 'data': {'input': {'messages': [HumanMessage(content='Thank you!! you are a great assistant. BTW, what i

You're welcome! I'm glad to be of help. However, I don't have the ability to remember or know your name unless you've told me in this conversation. If you'd like, you can tell me your name now!

# Summary

In this notebook, we have successfully implemented a multi-agent architecture utilizing LangGraph, marking a significant advancement in our capability to engineer complex, scalable systems. This framework enables the seamless integration of multiple agents, each performing distinct tasks, thereby facilitating more robust and intricate architectures.

**Note: Currently, this architecture does not include a memory component**, which is crucial for tasks that require historical data recall or context retention over time. However, the development of this memory component is underway and promises to enhance the functionality significantly. Once integrated, this memory capability will allow graphs to perform more complex tasks that require understanding and analyzing past interactions, leading to more intelligent and adaptive responses.

Stay tuned for updates!!